# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 1 2022 12:58PM,238551,19,GP06062022,"Granules Pharmaceuticals, Inc.",Released
1,Jun 1 2022 12:57PM,238542,10,0085732280,ISDIN Corporation,Released
2,Jun 1 2022 12:57PM,238542,10,0085732281,ISDIN Corporation,Released
3,Jun 1 2022 12:57PM,238542,10,0085728041,ISDIN Corporation,Released
4,Jun 1 2022 12:57PM,238542,10,0085732419,ISDIN Corporation,Released
5,Jun 1 2022 12:57PM,238542,10,0085732416,ISDIN Corporation,Released
6,Jun 1 2022 12:57PM,238542,10,0085732420,ISDIN Corporation,Released
7,Jun 1 2022 12:57PM,238542,10,0085732423,ISDIN Corporation,Released
8,Jun 1 2022 12:57PM,238542,10,0085732417,ISDIN Corporation,Released
9,Jun 1 2022 12:57PM,238542,10,0085732422,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
47,238546,Released,18
48,238547,Released,1
49,238548,Released,27
50,238549,Released,1
51,238551,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238546,NaN,NaN,18.0
238547,NaN,NaN,1.0
238548,NaN,NaN,27.0
238549,NaN,NaN,1.0
238551,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238318,0.0,13.0,7.0
238342,0.0,1.0,0.0
238343,0.0,0.0,1.0
238433,0.0,3.0,20.0
238436,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238318,0,13,7
238342,0,1,0
238343,0,0,1
238433,0,3,20
238436,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238318,0,13,7
1,238342,0,1,0
2,238343,0,0,1
3,238433,0,3,20
4,238436,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238318,,13,7
1,238342,,1,
2,238343,,,1
3,238433,,3,20
4,238436,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 1 2022 12:58PM,238551,19,"Granules Pharmaceuticals, Inc."
1,Jun 1 2022 12:57PM,238542,10,ISDIN Corporation
47,Jun 1 2022 12:56PM,238548,20,Alumier Labs Inc.
74,Jun 1 2022 12:53PM,238549,19,"Zymergen, Inc."
75,Jun 1 2022 12:51PM,238547,16,Sartorius Bioprocess Solutions
76,Jun 1 2022 12:48PM,238546,10,Beach Products Inc
94,Jun 1 2022 12:47PM,238544,20,"ACI Healthcare USA, Inc."
115,Jun 1 2022 12:46PM,238545,10,Emerginnova
118,Jun 1 2022 12:43PM,238541,10,"Senseonics, Incorporated"
119,Jun 1 2022 12:40PM,238540,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 1 2022 12:58PM,238551,19,"Granules Pharmaceuticals, Inc.",,,1
1,Jun 1 2022 12:57PM,238542,10,ISDIN Corporation,,,46
2,Jun 1 2022 12:56PM,238548,20,Alumier Labs Inc.,,,27
3,Jun 1 2022 12:53PM,238549,19,"Zymergen, Inc.",,,1
4,Jun 1 2022 12:51PM,238547,16,Sartorius Bioprocess Solutions,,,1
5,Jun 1 2022 12:48PM,238546,10,Beach Products Inc,,,18
6,Jun 1 2022 12:47PM,238544,20,"ACI Healthcare USA, Inc.",,,21
7,Jun 1 2022 12:46PM,238545,10,Emerginnova,,,3
8,Jun 1 2022 12:43PM,238541,10,"Senseonics, Incorporated",,,1
9,Jun 1 2022 12:40PM,238540,16,Sartorius Bioprocess Solutions,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 12:58PM,238551,19,"Granules Pharmaceuticals, Inc.",1,,
1,Jun 1 2022 12:57PM,238542,10,ISDIN Corporation,46,,
2,Jun 1 2022 12:56PM,238548,20,Alumier Labs Inc.,27,,
3,Jun 1 2022 12:53PM,238549,19,"Zymergen, Inc.",1,,
4,Jun 1 2022 12:51PM,238547,16,Sartorius Bioprocess Solutions,1,,
5,Jun 1 2022 12:48PM,238546,10,Beach Products Inc,18,,
6,Jun 1 2022 12:47PM,238544,20,"ACI Healthcare USA, Inc.",21,,
7,Jun 1 2022 12:46PM,238545,10,Emerginnova,3,,
8,Jun 1 2022 12:43PM,238541,10,"Senseonics, Incorporated",1,,
9,Jun 1 2022 12:40PM,238540,16,Sartorius Bioprocess Solutions,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 12:58PM,238551,19,"Granules Pharmaceuticals, Inc.",1,,
1,Jun 1 2022 12:57PM,238542,10,ISDIN Corporation,46,,
2,Jun 1 2022 12:56PM,238548,20,Alumier Labs Inc.,27,,
3,Jun 1 2022 12:53PM,238549,19,"Zymergen, Inc.",1,,
4,Jun 1 2022 12:51PM,238547,16,Sartorius Bioprocess Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 12:58PM,238551,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
1,Jun 1 2022 12:57PM,238542,10,ISDIN Corporation,46.0,NaN,NaN
2,Jun 1 2022 12:56PM,238548,20,Alumier Labs Inc.,27.0,NaN,NaN
3,Jun 1 2022 12:53PM,238549,19,"Zymergen, Inc.",1.0,NaN,NaN
4,Jun 1 2022 12:51PM,238547,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 12:58PM,238551,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
1,Jun 1 2022 12:57PM,238542,10,ISDIN Corporation,46.0,0.0,0.0
2,Jun 1 2022 12:56PM,238548,20,Alumier Labs Inc.,27.0,0.0,0.0
3,Jun 1 2022 12:53PM,238549,19,"Zymergen, Inc.",1.0,0.0,0.0
4,Jun 1 2022 12:51PM,238547,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3100719,156.0,11.0,9.0
102,953920,3.0,1.0,0.0
12,477037,60.0,2.0,0.0
15,1430558,52.0,50.0,0.0
16,1431210,6.0,1.0,0.0
18,715207,2.0,1.0,0.0
19,954120,12.0,11.0,8.0
20,1192586,69.0,4.0,6.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3100719,156.0,11.0,9.0
1,102,953920,3.0,1.0,0.0
2,12,477037,60.0,2.0,0.0
3,15,1430558,52.0,50.0,0.0
4,16,1431210,6.0,1.0,0.0
5,18,715207,2.0,1.0,0.0
6,19,954120,12.0,11.0,8.0
7,20,1192586,69.0,4.0,6.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,156.0,11.0,9.0
1,102,3.0,1.0,0.0
2,12,60.0,2.0,0.0
3,15,52.0,50.0,0.0
4,16,6.0,1.0,0.0
5,18,2.0,1.0,0.0
6,19,12.0,11.0,8.0
7,20,69.0,4.0,6.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,156.0
1,102,Released,3.0
2,12,Released,60.0
3,15,Released,52.0
4,16,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20
Status,,,,,,,,
Completed,9.0,0.0,0.0,0.0,0.0,0.0,8.0,6.0
Executing,11.0,1.0,2.0,50.0,1.0,1.0,11.0,4.0
Released,156.0,3.0,60.0,52.0,6.0,2.0,12.0,69.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0,8.0,6.0
1,Executing,11.0,1.0,2.0,50.0,1.0,1.0,11.0,4.0
2,Released,156.0,3.0,60.0,52.0,6.0,2.0,12.0,69.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0,8.0,6.0
1,Executing,11.0,1.0,2.0,50.0,1.0,1.0,11.0,4.0
2,Released,156.0,3.0,60.0,52.0,6.0,2.0,12.0,69.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()